In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import os


In [2]:
dataset_path = os.listdir("DATASET/TRAIN")

In [3]:
label_types = os.listdir("DATASET/TRAIN")
print(label_types)

['Anomaly', 'Bad Footage', 'Empty', 'Full', 'Normal']


# PREPARING TRAINING DATA

In [6]:
data1= []

for item in dataset_path:
    # Getting file names
    all_data = os.listdir("DATASET/TRAIN" + "/"+item)

    # Adding them in the list 
    for data in all_data:
        data1.append((item,str("DATASET/TRAIN" + "/"+item) + "/" + data))

# Building dataframe 
train_df = pd.DataFrame(data = data1,columns = ['tag','video_name'])
print(train_df.head())
print(train_df.tail())

       tag                                video_name
0  Anomaly  DATASET/TRAIN/Anomaly/v001_converted.mp4
1  Anomaly  DATASET/TRAIN/Anomaly/v002_converted.mp4
2  Anomaly  DATASET/TRAIN/Anomaly/v003_converted.mp4
3  Anomaly  DATASET/TRAIN/Anomaly/v004_converted.mp4
4  Anomaly  DATASET/TRAIN/Anomaly/v006_converted.mp4
        tag                                      video_name
346  Normal  DATASET/TRAIN/Normal/chunk_Cam_3_Feb 22_25.mp4
347  Normal  DATASET/TRAIN/Normal/chunk_Cam_3_Feb 22_26.mp4
348  Normal   DATASET/TRAIN/Normal/chunk_Cam_3_Feb 22_6.mp4
349  Normal   DATASET/TRAIN/Normal/chunk_Cam_3_Feb 22_7.mp4
350  Normal   DATASET/TRAIN/Normal/chunk_Cam_3_Feb 22_8.mp4


In [7]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv("train.csv")

# PREPARING TEST DATA 

In [8]:
dataset_path = os.listdir('DATASET/TEST')
print(dataset_path)

['Anomaly', 'Bad Footage', 'Empty', 'Full', 'Normal']


In [12]:
label_types = os.listdir("DATASET/TEST")
print("Types of activities found :",len(dataset_path))

Types of activities found : 5


In [16]:
data1= []

for item in dataset_path:
    # Getting file names
    all_data = os.listdir("DATASET/TEST" + "/"+item)

    # Adding them in the list 
    for data in all_data:
        data1.append((item,str("DATASET/TEST" + "/"+item) + "/" + data))

# Building dataframe 
test_df = pd.DataFrame(data = data1 ,columns = ['tag','video_name'])
print(test_df.head())
print(test_df.tail())

       tag                               video_name
0  Anomaly  DATASET/TEST/Anomaly/v080_converted.mp4
1  Anomaly  DATASET/TEST/Anomaly/v081_converted.mp4
2  Anomaly  DATASET/TEST/Anomaly/v082_converted.mp4
3  Anomaly  DATASET/TEST/Anomaly/v083_converted.mp4
4  Anomaly  DATASET/TEST/Anomaly/v084_converted.mp4
       tag                                     video_name
85  Normal          DATASET/TEST/Normal/chunk_Cam3_17.mp4
86  Normal          DATASET/TEST/Normal/chunk_Cam3_18.mp4
87  Normal          DATASET/TEST/Normal/chunk_Cam3_19.mp4
88  Normal  DATASET/TEST/Normal/chunk_Cam_3_Feb 22_27.mp4
89  Normal  DATASET/TEST/Normal/chunk_Cam_3_Feb 22_28.mp4


In [17]:
df = test_df.loc[:,['video_name','tag']]
df
df.to_csv("test.csv")

In [31]:
# !pip install git+https://github.com/tensorflow/docs

In [4]:

from tensorflow_docs.vis import embed
import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 351
Total videos for testing: 90


,Unnamed: 0,video_name,tag
106,106,DATASET/TRAIN/Bad Footage/chunk_Cam_3_Feb 11_8...,Bad Footage
79,79,DATASET/TRAIN/Bad Footage/chunk_Cam_3_Feb 11_1...,Bad Footage
270,270,DATASET/TRAIN/Normal/chunk_Cam_1_Feb 18_20.mp4,Normal
190,190,DATASET/TRAIN/Empty/chunk_Cam_2_Feb 18_10.mp4,Empty
256,256,DATASET/TRAIN/Normal/chunk_Cam_1_Feb 11_25.mp4,Normal
126,126,DATASET/TRAIN/Empty/chunk_Cam1_6.mp4,Empty
103,103,DATASET/TRAIN/Bad Footage/chunk_Cam_3_Feb 11_5...,Bad Footage
328,328,DATASET/TRAIN/Normal/chunk_Cam_3_Feb 18_24.mp4,Normal
220,220,DATASET/TRAIN/Full/chunk_Cam_1_Feb 11_26.mp4,Full
149,149,DATASET/TRAIN/Empty/chunk_Cam2_8.mp4,Empty


In [7]:

IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

# TO FEED THE VIDEO PERFORMING PROCESSING TASK :

In [7]:

IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

# FEATURE EXTRACTION

In [57]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

# LABEL ENCODING 

In [58]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Anomaly', 'Bad Footage', 'Empty', 'Full', 'Normal']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [59]:
print(train_data[0].shape)
train_data[0]


(351, 30, 2048)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [68]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 8
EPOCHS = 100

MAX_SEQ_LENGTH = 30
NUM_FEATURES = 2048

In [69]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")


Frame features in train set: (351, 30, 2048)
Frame masks in train set: (351, 30)
train_labels in train set: (351, 1)
test_labels in train set: (90, 1)


In [70]:
# `frame_masks` and `frame_features` are what we will feed to our sequence model.
# `frame_masks` will contain a bunch of booleans denoting if a timestep is
# masked with padding or not.

# SEQUENCE MODEL 

In [79]:
EPOCHS = 50
# Utility for running experiments.
def run_experiment():
    filepath = "./model/video_classifier.weights.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100,2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3027 - loss: 1.6086
Epoch 1: val_loss improved from inf to 1.61577, saving model to ./model/video_classifier.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 213ms/step - accuracy: 0.3116 - loss: 1.6083 - val_accuracy: 0.0000e+00 - val_loss: 1.6158
Epoch 2/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3615 - loss: 1.6046
Epoch 2: val_loss did not improve from 1.61577
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3719 - loss: 1.6041 - val_accuracy: 0.0000e+00 - val_loss: 1.6227
Epoch 3/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.3665 - loss: 1.6007
Epoch 3: val_loss did not improve from 1.61577
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3757 - loss: 1.6000 - val_accuracy: 0.0000e+00 - val_loss: 1.6298
Epoch 4/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4095 - loss: 1.5938
Epoch 4: val_loss did not improve from 1.61577
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4092 - loss: 1.5937 

# INFERENCE 

In [40]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: DATASET/TEST/Normal/chunk_Cam2_11.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 993ms/step
  Empty: 20.18%
  Anomaly: 20.17%
  Bad Footage: 19.90%
  Full: 19.90%
  Normal: 19.87%


In [22]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="DATASET/TEST/Anomaly/v098_converted.mp4
        type="video/mp4" style="height:300px;width:300px">
    </video>
""")